In [1]:
from tqdm import tqdm
import pandas as pd

In [2]:
import numpy as np

In [3]:
df_test = pd.read_csv('data/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

In [4]:
names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))

In [5]:
# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle

In [6]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

In [7]:
rles = []

In [8]:
saved_horiz_preds_start = None

In [9]:
saved_vert_preds_start = None

In [10]:
for start in tqdm(range(0, len(ids_test), 10)):
    vert_batch_start = int(start / 240)*240
    horiz_batch_start  = int(start / 250)*250
    
    if saved_vert_preds_start == vert_batch_start:
        vert_preds = saved_vert_preds
    else:
        vert_preds = load_array('./vert-cut-preds/batch-' + str(vert_batch_start))
        saved_vert_preds = vert_preds
        saved_vert_preds_start = vert_batch_start
    
    if saved_horiz_preds_start == horiz_batch_start:
        horiz_top_preds = saved_horiz_top_preds
        horiz_bot_preds = saved_horiz_bot_preds
    else:
        horiz_top_preds = load_array('./horiz-cut-top-preds/batch-'+str(horiz_batch_start))
        horiz_bot_preds = load_array('./horiz-cut-bottom-preds/batch-'+str(horiz_batch_start))
        saved_horiz_top_preds = horiz_top_preds
        saved_horiz_bot_preds = horiz_bot_preds
        saved_horiz_preds_start = horiz_batch_start
        
    
    vert_within_slnos  = list(range(start-vert_batch_start,  min(len(vert_preds), start-vert_batch_start+10)))
    horiz_within_slnos = list(range(start-horiz_batch_start, min(len(horiz_top_preds), start-horiz_batch_start+10)))
    
    vert_preds = vert_preds[vert_within_slnos, :, :]
    horiz_top_preds = horiz_top_preds[horiz_within_slnos, :, :]
    horiz_bot_preds = horiz_bot_preds[horiz_within_slnos, :, :]
    
    horiz_top_preds = horiz_top_preds[:,:,2:]
    horiz_bot_preds = horiz_bot_preds[:,:,0:1918]
    
    part_1   = horiz_top_preds[:,0:512,:]
    part_2_1 = horiz_top_preds[:,512:,:]
    part_2_2 = horiz_bot_preds[:,0:256,:]
    part_3   = horiz_bot_preds[:,256:,:]
    part_2   = np.add(part_2_1, part_2_2)/2
    horiz_preds = np.concatenate((part_1, part_2, part_3), axis=1)
    
    avg_preds = np.add(horiz_preds, vert_preds)/2
    
    avg_preds = avg_preds > 50
    
    for i in range(avg_preds.shape[0]):
        mask = np.copy(avg_preds[i,:,:])
        rle = run_length_encode(mask)
        rles.append(rle)

100%|█████████▉| 10006/10007 [1:13:29<00:00,  2.47it/s]

IndexError: index 224 is out of bounds for axis 0 with size 224

In [18]:
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission16.csv.gz', index=False, compression='gzip')

In [8]:
rles = []
saved_horiz_preds_start = None
saved_vert_preds_start = None

In [9]:
for start in tqdm(range(0, len(ids_test), 10)):
    vert_batch_start = int(start / 240)*240
    horiz_batch_start  = int(start / 250)*250
    
    if saved_vert_preds_start == vert_batch_start:
        vert_preds = saved_vert_preds
    else:
        vert_preds = load_array('./vert-cut-preds/batch-' + str(vert_batch_start))
        saved_vert_preds = vert_preds
        saved_vert_preds_start = vert_batch_start
    
    if saved_horiz_preds_start == horiz_batch_start:
        horiz_top_preds = saved_horiz_top_preds
        horiz_bot_preds = saved_horiz_bot_preds
    else:
        horiz_top_preds = load_array('./horiz-cut-top-preds/batch-'+str(horiz_batch_start))
        horiz_bot_preds = load_array('./horiz-cut-bottom-preds/batch-'+str(horiz_batch_start))
        saved_horiz_top_preds = horiz_top_preds
        saved_horiz_bot_preds = horiz_bot_preds
        saved_horiz_preds_start = horiz_batch_start
        
    
    vert_within_slnos  = list(range(start-vert_batch_start,  min(len(vert_preds), start-vert_batch_start+10)))
    horiz_within_slnos = list(range(start-horiz_batch_start, min(len(horiz_top_preds), start-horiz_batch_start+10)))
    
    vert_preds = vert_preds[vert_within_slnos, :, :]
    horiz_top_preds = horiz_top_preds[horiz_within_slnos, :, :]
    horiz_bot_preds = horiz_bot_preds[horiz_within_slnos, :, :]
    
    horiz_top_preds = horiz_top_preds[:,:,2:]
    horiz_bot_preds = horiz_bot_preds[:,:,0:1918]
    
    part_1   = horiz_top_preds[:,0:512,:]
    part_2_1 = horiz_top_preds[:,512:,:]
    part_2_2 = horiz_bot_preds[:,0:256,:]
    part_3   = horiz_bot_preds[:,256:,:]
    part_2   = np.add(part_2_1, part_2_2)/2
    horiz_preds = np.concatenate((part_1, part_2, part_3), axis=1)
    
    horiz_preds = horiz_preds > 50
    vert_preds  = vert_preds > 50
    avg_preds   = np.logical_and(horiz_preds, vert_preds)
    
    for i in range(avg_preds.shape[0]):
        mask = np.copy(avg_preds[i,:,:])
        rle = run_length_encode(mask)
        rles.append(rle)

100%|██████████| 10007/10007 [48:39<00:00,  3.43it/s] 


In [10]:
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission17.csv.gz', index=False, compression='gzip')

In [7]:
rles = []
saved_horiz_preds_start = None
saved_vert_preds_start = None

In [8]:
for start in tqdm(range(0, len(ids_test), 10)):
    vert_batch_start = int(start / 240)*240
    horiz_batch_start  = int(start / 250)*250
    
    if saved_vert_preds_start == vert_batch_start:
        vert_preds = saved_vert_preds
    else:
        vert_preds = load_array('./vert-cut-preds/batch-' + str(vert_batch_start))
        saved_vert_preds = vert_preds
        saved_vert_preds_start = vert_batch_start
    
    if saved_horiz_preds_start == horiz_batch_start:
        horiz_top_preds = saved_horiz_top_preds
        horiz_bot_preds = saved_horiz_bot_preds
    else:
        horiz_top_preds = load_array('./horiz-cut-top-preds/batch-'+str(horiz_batch_start))
        horiz_bot_preds = load_array('./horiz-cut-bottom-preds/batch-'+str(horiz_batch_start))
        saved_horiz_top_preds = horiz_top_preds
        saved_horiz_bot_preds = horiz_bot_preds
        saved_horiz_preds_start = horiz_batch_start
        
    
    vert_within_slnos  = list(range(start-vert_batch_start,  min(len(vert_preds), start-vert_batch_start+10)))
    horiz_within_slnos = list(range(start-horiz_batch_start, min(len(horiz_top_preds), start-horiz_batch_start+10)))
    
    vert_preds = vert_preds[vert_within_slnos, :, :]
    horiz_top_preds = horiz_top_preds[horiz_within_slnos, :, :]
    horiz_bot_preds = horiz_bot_preds[horiz_within_slnos, :, :]
    
    horiz_top_preds = horiz_top_preds[:,:,2:]
    horiz_bot_preds = horiz_bot_preds[:,:,0:1918]
    
    part_1   = horiz_top_preds[:,0:512,:]
    part_2_1 = horiz_top_preds[:,512:,:]
    part_2_2 = horiz_bot_preds[:,0:256,:]
    part_3   = horiz_bot_preds[:,256:,:]
    part_2   = np.add(part_2_1, part_2_2)/2
    horiz_preds = np.concatenate((part_1, part_2, part_3), axis=1)
    
    horiz_preds = horiz_preds > 50
    vert_preds  = vert_preds > 50
    avg_preds   = np.logical_or(horiz_preds, vert_preds)
    
    for i in range(avg_preds.shape[0]):
        mask = np.copy(avg_preds[i,:,:])
        rle = run_length_encode(mask)
        rles.append(rle)

100%|██████████| 10007/10007 [48:48<00:00,  3.42it/s] 


In [9]:
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('submit/submission18.csv.gz', index=False, compression='gzip')